In [1]:
pip install tensorflow==2.11.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 30.9 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninsta

In [2]:
import tensorflow as tf
print(tf.__version__)


2.11.0


In [3]:
# Import necessary libraries
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from keras.optimizers import RMSprop
import random
import tensorflow as tf
from tensorflow import keras
import numpy as np
from glob import glob
import matplotlib.pyplot as plt



In [4]:
IMAGE_SIZE = [224,224]
TrainFolder = "/kaggle/input/resnetv2/datasetv2/train"
ValidateFolder = "/kaggle/input/resnetv2/datasetv2/val"



In [5]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
keras.utils.set_random_seed(42)

In [6]:
myResnet = ResNet50(input_shape= IMAGE_SIZE+[3] , weights='imagenet', include_top=False)

print(myResnet.summary())

94765736/94765736 [==============================] - 4s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                          

In [7]:
for layer in myResnet.layers:
    layer.trainable = False

Classes = glob('/kaggle/input/resnetv2/datasetv2/train/*')

print(Classes)

['/kaggle/input/resnetv2/datasetv2/train/Golongan2', '/kaggle/input/resnetv2/datasetv2/train/Golongan1']


In [8]:
classesNum = len(Classes)
print(classesNum)

2


In [9]:
PlusFlattenlayer = Flatten() (myResnet.output)

prediction = Dense(classesNum, activation='softmax')(PlusFlattenlayer)

model = Model(inputs=myResnet.input, outputs = prediction)

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [10]:
from keras.optimizers import RMSprop
from keras.optimizers import Adam

# Create an instance of RMSprop with a custom learning rate
optimizer = Adam(learning_rate=0.001)

# Compile the model with the custom optimizer
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

print(f"Current learning rate: {optimizer.learning_rate.numpy()}")
print(f"Current optimizer: {model.optimizer}")
print(f"Optimizer configuration: {model.optimizer.get_config()}")


Current learning rate: 0.0010000000474974513
Current optimizer: <keras.optimizers.optimizer_v2.adam.Adam object at 0x7830dc1459f0>
Optimizer configuration: {'name': 'Adam', 'learning_rate': 0.001, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}


In [11]:
from keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(rescale= 1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:

training_set = train_datagen.flow_from_directory(TrainFolder, target_size=(224,224), batch_size=32, class_mode='categorical')
test_set = test_datagen.flow_from_directory(ValidateFolder, target_size=(224,224),batch_size=32, class_mode='categorical')

result = model.fit(training_set, validation_data=test_set, epochs=100, steps_per_epoch=len(training_set), validation_steps=len(test_set))

Found 418 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Epoch 1/100
14/14 [==============================] - 55s 4s/step - loss: 3.7767 - accuracy: 0.5742 - val_loss: 2.3707 - val_accuracy: 0.3500
Epoch 2/100
14/14 [==============================] - 49s 4s/step - loss: 1.1180 - accuracy: 0.6603 - val_loss: 0.7183 - val_accuracy: 0.7500
Epoch 3/100
14/14 [==============================] - 58s 4s/step - loss: 0.2524 - accuracy: 0.8947 - val_loss: 0.1073 - val_accuracy: 0.9667
Epoch 4/100
14/14 [==============================] - 53s 4s/step - loss: 0.1173 - accuracy: 0.9593 - val_loss: 0.1149 - val_accuracy: 0.9500
Epoch 5/100
14/14 [==============================] - 49s 4s/step - loss: 0.0974 - accuracy: 0.9641 - val_loss: 0.1859 - val_accuracy: 0.9167
Epoch 6/100
14/14 [==============================] - 54s 4s/step - loss: 0.1084 - accuracy: 0.9522 - val_loss: 0.1062 - val_accuracy: 0.9500
Epoch 7/100
14/14 [==============================] - 49s 4s/step - loss: 

In [13]:
model.save('/kaggle/working/HasilModel/Adam100Batch32setv2lr0.001.h5')